# Recomendador basado en contenidos
Consiste en recomendar contenido similar a un determinado contenido

#### Principales caracteristicas
- Sólo necesita la información de un producto, por lo que no se necesita información histórica
- Se suele usar para evitar problemas de arranque en frio

#### Como lo hacemos
1. Definir una métrica de similitud para comparar los contenidos
2. Calcular la similitud entre todos los elementos con la métrica seleccionada
3. (Opcional) Definir vecindarios para optimizar


## Antes de comenzar... veamos el dataset
Recursos turisticos de Andalucia obtenidos de [TripAdvisor](https://www.tripadvisor.es)

In [1]:
import pandas as pd

recursos = pd.read_csv('./recursos/Recursos_turisticos.csv', sep='|')
recursos.head(5)

,_id,nombre_ws,strCategoria,strSubcategoria,strCiudad,strDescripcion
0,5880b148636e564ad8d9b952,Hotel Albucasis,Alojamientos,Hoteles,Córdoba,.El hotel tiene a disposición de sus clientes ...
1,5880b148636e564ad8d9b963,Hotel Sierra Hidalga,Alojamientos,Hoteles,Ronda,Enclavado en una situación privilegiada con fa...
2,5880b1f2636e564ad8d9b969,Hotel Virgen de los Reyes,Alojamientos,Hoteles,Sevilla,"El Hotel Virgen de los Reyes***, ubicado en el..."
3,5880b1f2636e564ad8d9b96a,Hotel An Santo Domingo,Alojamientos,Hoteles,Lucena,El Hotel Santo Domingo Lucena está situado en ...
4,5880b1f2636e564ad8d9b96f,Leflet Valme,Alojamientos,Hoteles,Dos Hermanas,El Hotel Leflet Valme está situado estratégica...


## Metrica de similitud para texto

Una de las técnicas más utilizadas para obtener similitude texto es TF-IDF, tecnica proveniente del campo de Information Retrieval.

TF-IDF permite realizar una representación vectorial de los textos. Por ejemplo:

> "En un lugar de la mancha"

|_|En|un |lugar|de|la|mancha|
|--|--|---|-----|--|--|------|
|TF |1|1|1|1|1|1|
|DF |200|1700|100|3000|4075|3|
|IDF|1/200|1/1700|1/100|1/3000|1/4075|1/3|

Al tener una representación vectorial de los textos podemos usar medidas tradicionales
<img src="./recursos/similitudes.png">

### Ejemplo TF-IDF

**Paso 1** Preparacion y tokenizado de datos

In [2]:
import gensim
import nltk
import tempfile
import heapq

from nltk.tokenize import word_tokenize
nltk.download('punkt')

documentos = recursos['strDescripcion']
print ("Numero de documentos:", len(documentos))
print ("\n", documentos[1])
       
docs_tokenizado =[[w.lower() for w in word_tokenize(text)] 
            for text in documentos]

print ("\n\nElementos tokenizados\n")
print (docs_tokenizado[1])


[nltk_data] Downloading package punkt to /Users/Mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Numero de documentos: 601

 Enclavado en una situación privilegiada con fantásticas vistas a la sierra, y a menos de 2 Km. del centro de la ciudad de Ronda y 45 Km. de la Costa del Sol, es un lugar ideal para disfrutar de la oferta cultural y gastronómica además del patrimonio histórico-artístico de la ciudad, pudiendo combinarla con actividades en su entorno natural como la Sierra de las Nieves (reserva de la biosfera). Su decoración de estilo moderno y actual ofrece un ambiente selecto y acogedor. En nuestro restaurante podrán degustar los platos de nuestro cocinero Antonio Castro Aguilar además de una exquisita carta de vinos..


Elementos tokenizados

['enclavado', 'en', 'una', 'situación', 'privilegiada', 'con', 'fantásticas', 'vistas', 'a', 'la', 'sierra', ',', 'y', 'a', 'menos', 'de', '2', 'km', '.', 'del', 'centro', 'de', 'la', 'ciudad', 'de', 'ronda', 'y', '45', 'km', '.', 'de', 'la', 'costa', 'del', 'sol', ',', 'es', 'un', 'lugar', 'ideal', 'para', 'disfrutar', 'de', 'la', 'o

**Paso 2** Creación de un un diccionario con todo el vocabulario

In [3]:
diccionario = gensim.corpora.Dictionary(docs_tokenizado)

print("Número de palabras en el diccionario:",len(diccionario))
for i in range(len(diccionario)):
    print(i, diccionario[i])

Número de palabras en el diccionario: 7965
0 ,
1 .
2 .el
3 100
4 3
5 30
6 a
7 albolafia
8 amantes
9 antiguo
10 aves
11 barrio
12 belleza
13 bus
14 calle
15 cama
16 ciudad
17 clientes
18 coche
19 cual
20 cuna
21 córdoba
22 de
23 del
24 describir
25 diferentes
26 disposición
27 donde
28 en
29 encuentra
30 especies
31 esta
32 guadalquivir
33 hermosa
34 hotel
35 información
36 judería
37 la
38 los
39 mezquita
40 minutos
41 monumentos
42 morena
43 muchos
44 museos
45 más
46 naturaleza
47 no
48 nuestra
49 o
50 observar
51 oficina
52 para
53 pelo
54 pie
55 poco
56 posee
57 puede
58 pueden
59 que
60 recomendamos
61 relajarnos
62 restaurantes
63 ribera
64 rincón
65 rio
66 se
67 secador
68 sierra
69 situados
70 sotos
71 supletoria
72 sus
73 tabernas
74 tanta
75 televisión
76 tiene
77 todo
78 tranquilo
79 turismo
80 turística.estamos
81 típica
82 un
83 una
84 visitar
85 y
86 (
87 )
88 2
89 45
90 acogedor
91 actividades
92 actual
93 además
94 aguilar
95 ambiente
96 antonio
97 biosfera
98 carta
99 

981 posible
982 relación
983 satisfecho
984 sienta
985 tapas
986 teatro
987 tendrá
988 6
989 agua
990 algo
991 alimentado
992 amor
993 artísticos
994 arómaticas
995 beneficios
996 cascada
997 cerrado
998 contamos
999 cortijo
1000 creado
1001 definitivamente
1002 descansar
1003 desconectar
1004 doñana
1005 e
1006 ecológico
1007 encontrar
1008 exteriores
1009 frutales
1010 huelva
1011 huerto
1012 inolvidable
1013 japonés
1014 jardines.tambien
1015 llega
1016 malvasias
1017 malvasías
1018 medicinales
1019 mesa
1020 muchas
1021 naturaleza.contamos
1022 númerosos
1023 originales
1024 pocos
1025 poder
1026 podrás
1027 puedas
1028 pájaros
1029 relajarte
1030 respeto
1031 sabores
1032 sinfín
1033 sonido
1034 temáticamente
1035 tropicales
1036 vienen
1037 visitarnos
1038 árboles
1039 único
1040 don
1041 miguel
1042 abuelo
1043 almuñécar
1044 aparcamiento
1045 barbacoas
1046 baza.las
1047 clima
1048 climatizada.entre
1049 continental
1050 cuestión
1051 cumbres
1052 diócesis
1053 doce
1054 formad

1731 sitúa
1732 villanueva
1733 villas
1734 visitas
1735 ¡damos
1736 a49
1737 alcázares
1738 autopista
1739 bormujos
1740 cercana
1741 desplazarse
1742 domocenter
1743 empresarial
1744 equipada
1745 estudios
1746 famosos
1747 hallan
1748 llaves
1749 pasear
1750 reales
1751 red
1752 rápidamente
1753 sevilla-huelva
1754 transporte
1755 une
1756 2014
1757 9
1758 acondicionado/calefaccion
1759 alcazares
1760 articulos
1761 aseos
1762 constan
1763 corazon
1764 cruz
1765 diciembre
1766 doña
1767 emblematicos
1768 fi
1769 hosteria
1770 lina
1771 localizacion
1772 televisor
1773 wi-
1774 acondicionadocalefacciónescritoriohabitaciones
1775 andaluces
1776 completoaire
1777 cortijos
1778 descubre
1779 especialistas
1780 espárrago
1781 granadino
1782 gratuitoparking
1783 hr
1784 huétor
1785 mediterránea.las
1786 moderna
1787 modernidad.wifi
1788 morado
1789 nacionales
1790 peticiónamenities
1791 poniente
1792 regionales
1793 rural.capacidad
1794 rústico
1795 supletoriascunas
1796 tajar
1797 tranqu

2480 perteneciente
2481 portátil
2482 prensa
2483 primera.alcalá
2484 provincias
2485 torrepalma
2486 abre
2487 gaditanos
2488 maría
2489 paisajes
2490 visite
2491 autobus
2492 btt
2493 carrtera
2494 dispuestas
2495 funcionales
2496 mirasol
2497 motril
2498 multitud
2499 quads
2500 sencilla
2501 tienes
2502 trayecto
2503 órgiva
2504 caracteriza
2505 cuidada
2506 marco
2507 predomina
2508 santillán
2509 67
2510 acceder
2511 adicional
2512 coste
2513 cuádruples
2514 dejara
2515 desea
2516 destacan
2517 dream
2518 económico
2519 escapada
2520 especialmente
2521 esquiar
2522 ofrecerá
2523 sencillez
2524 solárium
2525 supermercados
2526 urban
2527 actos
2528 café-bar
2529 casero
2530 climatizados
2531 compartir
2532 conferencias
2533 contiguos
2534 cozazón
2535 cuidado
2536 deseadas
2537 distintos
2538 equipamiento
2539 espada
2540 estacias
2541 gustoso
2542 hacerlas
2543 incomparable
2544 personaliza
2545 provistas
2546 proximidad
2547 renunciar
2548 rústica
2549 sociales
2550 unen
2551 un

3230 emplazamiento
3231 mil
3232 necesitas
3233 posada
3234 setecientos
3235 2009
3236 61
3237 ana
3238 completar
3239 decorado
3240 equipajes.para
3241 exclusividad
3242 firma
3243 histórico.cuenta
3244 juzgados
3245 prestigiosa
3246 princesa
3247 valenty.situado
3248 emblemática
3249 escapadas
3250 falte
3251 personalizados
3252 preparados
3253 sidonia
3254 wellness
3255 activa
3256 adapta
3257 complementarán
3258 cosigna
3259 embajador
3260 hostelería
3261 intermodal
3262 mágica
3263 necesiten
3264 numerosos
3265 precises
3266 quieran
3267 tapa
3268 tareas
3269 10km
3270 567
3271 administrativas
3272 aguadulce.el
3273 aguadulce.las
3274 aislamiento
3275 almerimarhabitacionesel
3276 almería•
3277 aparcamientos
3278 aquaerobic
3279 autóctonos
3280 bailes
3281 bancarias
3282 clases
3283 conserjería-recepción
3284 cristales
3285 deportiva
3286 descanso.centros
3287 deseen
3288 distribuidas
3289 diversión.el
3290 elaborados
3291 entidades
3292 envía
3293 estén
3294 heladerías
3295 km.
32

...

4978 sotogrande
4979 torno
4980 valderrama
4981 alegra
4982 azulejería
4983 corregidor
4984 enrejados
4985 esplendoroso
4986 expone
4987 florida
4988 mestizaje
4989 ocre
4990 plagado
4991 terrazos
4992 76
4993 alcazares.esta
4994 conexion
4995 estacion
4996 gratuitamente
4997 h10
4998 hotel.como
4999 justa.recientemente
5000 kms
5001 limitadas
5002 pelo.dispone
5003 satelite
5004 situtado
5005 tft-lcd
5006 tipico
5007 faciales.el
5008 gratuita.el
5009 local.el
5010 minibar.el
5011 precise
5012 rompido-
5013 barroco
5014 centros
5015 dolménico
5016 vitalizan
5017 2.00m
5018 amarga
5019 artesanías
5020 bañeras
5021 cajas
5022 característica
5023 carboneras
5024 diversidad
5025 dosel
5026 encantados
5027 esforzamos
5028 estaremos
5029 falta
5030 frescas
5031 gestionarlo
5032 jacusi
5033 kiko
5034 lavabos
5035 mira
5036 nijar
5037 nunca
5038 razón
5039 sublime
5040 tio
5041 ¡les
5042 adaptándose
5043 alimentación
5044 aprovecharse
5045 callejas
5046 cesta
5047 contrastan
5048 cuenco
5049 f

5728 estupendos
5729 inexplorados
5730 paradisíacas
5731 piragua
5732 49
5733 a-382
5734 a-4
5735 autopistas
5736 center****
5737 paddel
5738 clasificación
5739 merecida
5740 objetivo
5741 obtención
5742 obteniendose
5743 reinaugurado
5744 torreluz
5745 accesos
5746 asistencia
5747 disponibles
5748 mesa/sillas
5749 mini-market
5750 papeleras
5751 pensión
5752 públicos
5753 recibidor
5754 regímenes
5755 sanitaria
5756 socorristas
5757 tfno.
5758 valoradas
5759 bar-salón
5760 colgantes
5761 higuera
5762 panorámicos
5763 peluquería…
5764 pisos
5765 playacapricho
5766 potos
5767 presidido
5768 incluído
5769 multi-sport
5770 ****
5771 edades.nuestra
5772 900
5773 atestiguan
5774 encarnación
5775 img
5776 lucero
5777 metrosol
5778 parasol
5779 paños
5780 pensada
5781 transmite
5782 15.000
5783 623
5784 al-andalus
5785 azahar
5786 emerge
5787 frescura
5788 heliópolis
5789 integralmente
5790 irradiando
5791 luisa
5792 maria
5793 recrearse
5794 silken
5795 arrocería
5796 deseas
5797 exposicione

6467 alexander
6468 astronómica
6469 bike
6470 campo.dispone
6471 clientes.ponemos
6472 corosy
6473 desayunos.el
6474 descensos
6475 dias
6476 endemica
6477 epoca
6478 españa.cuenta
6479 festivos.el
6480 fondales
6481 fondales.hotel
6482 fotografia
6483 general
6484 geologicas
6485 google
6486 guias
6487 hectareas
6488 high
6489 holidays
6490 informacion
6491 mapshotel
6492 mariposas
6493 mecina
6494 mountain
6495 observacion
6496 observaciones
6497 orquestas
6498 primavera
6499 realizarlas
6500 salada.ubicado
6501 senderistas
6502 slopes
6503 snm.con
6504 solistas
6505 técnica
6506 virtual
6507 conocidas
6508 repetir
6509 alixares
6510 esmerada
6511 hotel.dotado
6512 visitados
6513 callejón
6514 diarias
6515 fundación
6516 heren
6517 raimundo
6518 trayectoria
6519 5*****
6520 condes
6521 confluyen
6522 encuadradas
6523 g.l.
6524 grandioso
6525 jienense
6526 joya
6527 joyas
6528 master
6529 rica
6530 torreón
6531 330
6532 calefacción/aire
6533 elevado
6534 fabuloso
6535 insuperables
65

7064 guisos
7065 hortalizas
7066 imprescindible
7067 magallanes
7068 marinero
7069 ortiguillas
7070 protagonistas
7071 remonta
7072 romanos
7073 sanluqueños
7074 tortillitas
7075 visigodos
7076 280m2
7077 calita
7078 coquetas
7079 dispositivos
7080 distribuídos
7081 enseres
7082 fuentebravía
7083 ip
7084 puntilla
7085 redes
7086 repartidos
7087 smart
7088 telefonía
7089 utilizados
7090 vistahermosa
7091 útil
7092 arcángel
7093 exelente
7094 fragancia
7095 fresca
7096 grandiosidad
7097 recovecos
7098 absorción
7099 adicionalmente
7100 amueblado
7101 cardiovascular
7102 chorro
7103 minfrigorífico
7104 tft
7105 viscolásticos
7106 éstas
7107 jazmines
7108 alcázar.junto
7109 grata
7110 habitaciones.acceso
7111 hombres
7112 hotel.opcional.-
7113 maría.dispone
7114 peluquería.ideal
7115 vinotecas.a
7116 1970
7117 46
7118 cambiar
7119 cruceros
7120 encargará
7121 folletos
7122 guiadas…
7123 jentoft
7124 limpiamos
7125 oro…
7126 vigilado
7127 alquilan
7128 bar-restaurante
7129 escaleras
7130 fa

7764 velázquez
7765 cercanias
7766 colaboran
7767 mismas
7768 méxico
7769 rato
7770 restaurante-grill
7771 100m2
7772 55m2
7773 75m2
7774 calahonda
7775 estupendo
7776 lavadora.el
7777 siesta
7778 .disponemos
7779 12€
7780 150m
7781 día.especialmente
7782 fuentepiedra
7783 plantas.posee
7784 salon.zona
7785 sofás-cama
7786 tv.pantalla
7787 118
7788 aprox
7789 banquetes/eventos
7790 portil
7791 portuguesa
7792 rafael
7793 renfe
7794 seducirán
7795 valsequillo
7796 vive
7797 america
7798 ciudadela
7799 patio-jardin
7800 1,2
7801 mona
7802 alcohólicas
7803 alcohólicas.en
7804 cocktails
7805 crocodile
7806 escritorio.en
7807 estacionalidad
7808 futbolín.ponemos
7809 garden
7810 helados.nuestro
7811 jardines.todas
7812 limitado
7813 provenzal
7814 ect
7815 espeto
7816 exóticas
7817 gastronomia
7818 manquita
7819 optando
7820 sardinas
7821 submarino
7822 39
7823 atraer
7824 deleitar
7825 estilosas
7826 bobastro
7827 ciclista
7828 conde
7829 embalses
7830 guadalteba
7831 transandalus
7832 com

**Paso 3** Generación del corpus de documentos, se calcula el número de veces que cada palabra aparece en el documento (TF)

In [4]:
corpus = [diccionario.doc2bow(gen_doc) for gen_doc in docs_tokenizado]
print(corpus[67])

[(1, 1), (22, 1), (23, 3), (28, 1), (34, 2), (59, 1), (85, 2), (100, 1), (104, 1), (135, 1), (158, 3), (207, 1), (210, 1), (240, 1), (349, 1), (375, 1), (674, 1), (789, 1), (1380, 1), (2262, 1), (2263, 1), (2264, 1), (2265, 1), (2266, 1), (2267, 1)]


**Paso 4** Generación del modelo para TF-IDF

In [5]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)
tf_idf[corpus[45]]

TfidfModel(num_docs=601, num_nnz=54889)
54889


[(0, 0.008139695672173363),
 (1, 0.004510516788463085),
 (5, 0.08456603497118027),
 (6, 0.03000096365578004),
 (16, 0.03504409366605791),
 (22, 0.0024847904756781325),
 (23, 0.007694763140391543),
 (28, 0.007138366440110438),
 (34, 0.006439607744825614),
 (37, 0.025856743480079238),
 (52, 0.07093896007159398),
 (68, 0.11776675125644373),
 (77, 0.03862778197622343),
 (84, 0.07775965294352788),
 (85, 0.00709502122538406),
 (86, 0.038088623363609526),
 (87, 0.038088623363609526),
 (103, 0.05924269143515135),
 (104, 0.011783238035294777),
 (118, 0.10378052688500226),
 (124, 0.05924389845457021),
 (135, 0.08296142400852274),
 (139, 0.056146478779426495),
 (157, 0.04188264743184512),
 (158, 0.014062273585481108),
 (196, 0.11496691505019903),
 (232, 0.0497063272066019),
 (237, 0.06615409479008251),
 (240, 0.041929142628367964),
 (242, 0.12496038804156541),
 (250, 0.058527170544351276),
 (252, 0.07166850003453945),
 (255, 0.05647513150467492),
 (260, 0.0497063272066019),
 (265, 0.0615039297335

**Paso 5** Creación de un índice temporal

In [6]:
print (tempfile.gettempdir())
sims = gensim.similarities.Similarity(tempfile.gettempdir(),tf_idf[corpus],
                                      num_features=len(diccionario))
print (sims[tf_idf[corpus[45]]])

/var/folders/w7/6p2jts811bg78fk9prk3ppr40000gn/T
[3.46589647e-02 4.94005792e-02 8.65396671e-03 1.63417459e-02
 6.03248887e-02 1.50335003e-02 4.00168449e-02 7.68792108e-02
 6.83512830e-04 3.66436020e-02 5.04549965e-02 1.45706251e-01
 1.18437540e-02 9.74706747e-03 2.57664789e-02 5.18195443e-02
 5.50350808e-02 9.58429929e-03 2.08368525e-02 4.61855700e-04
 4.34412286e-02 2.38053538e-02 5.17604351e-02 3.73193398e-02
 9.28766504e-02 9.17262733e-02 6.23373799e-02 7.04123601e-02
 3.72223835e-03 1.42990667e-02 4.99481373e-02 2.27083229e-02
 3.49151529e-02 3.79720214e-03 6.38883635e-02 4.06165421e-02
 4.42574024e-02 5.69383577e-02 4.53083217e-02 5.68707287e-02
 9.32421628e-03 2.56028473e-02 6.26740158e-02 5.41344397e-02
 2.02719253e-02 1.00000024e+00 3.61943468e-02 6.90728948e-02
 2.10251994e-02 2.03605220e-02 3.01017165e-02 3.79155669e-03
 4.91253063e-02 6.29852638e-02 3.58827040e-02 3.58060673e-02
 2.14371905e-02 2.11800449e-02 1.83537044e-02 3.76007818e-02
 1.24543486e-02 4.40920666e-02 2.755

**Paso 6** Obtener elementos recomendados, los más similares

In [7]:
salida = sims[tf_idf[corpus[45]]]

heap = []
indice = 0
for medida in salida:
    heapq.heappush(heap, (medida, documentos[indice]))
    indice +=1

print (documentos[45])
heapq.nlargest(10,heap)

Situada en el pueblo de La Zubia, junto al Parque Natural de Sierra Nevada y tan solo 5 min de la capital nazarí. Ideal tanto para hacer excursiones y practicar senderismo como para visitar La Alhambra y la ciudad. Fácil acceso por autovía A- 92 10 min a La Alhambra, 20 min a la estación de esquí de Sierra Nevada, 25 a la Alpujarra y a 30 la playa. Chimenea para relajarse al calor de la lumbre, salón amplio, dividido en dos ambientes, con ventana de acceso a la cocina y tv grande (ideal para reuniones grandes). La cocina dispone de todo lo necesario, patio andaluz, 12 dormitorios con mucha luz. Disponemos de restaurante especializado en carnes a la brasa, tanto el hotel como el restaurante están rodeados de jardines para tomar el sol en cualquier época del año.


[(1.0000002,
  'Situada en el pueblo de La Zubia, junto al Parque Natural de Sierra Nevada y tan solo 5 min de la capital nazarí. Ideal tanto para hacer excursiones y practicar senderismo como para visitar La Alhambra y la ciudad. Fácil acceso por autovía A- 92 10 min a La Alhambra, 20 min a la estación de esquí de Sierra Nevada, 25 a la Alpujarra y a 30 la playa. Chimenea para relajarse al calor de la lumbre, salón amplio, dividido en dos ambientes, con ventana de acceso a la cocina y tv grande (ideal para reuniones grandes). La cocina dispone de todo lo necesario, patio andaluz, 12 dormitorios con mucha luz. Disponemos de restaurante especializado en carnes a la brasa, tanto el hotel como el restaurante están rodeados de jardines para tomar el sol en cualquier época del año.'),
 (0.17314425,
  'Nuestro hotel Alicia Carolina está situado en un lugar estratégico, tanto para visitar el centro histórico de Granada y la Alhambra (7 min en coche), como adentrarse en el maravilloso parque N

## Nuevas técnicas, deep learning

Es una nueva técnica dentro del aprendizaje automático, basado en la estructura de las redes neuronales, pero usando arquitecturas profundas (deep) en la que múltiples capas son conectadas.

La principal característica y ventaja del deep learning es el **feature discovering** sin la necesidad de la intervención humana.

### Paragraph vector

- Algoritmo de aprendizaje no supervisado capaz de generar una representacion vectorial a partir de texto.
- Paragraph vector es capaz de aprender la semántica de las palabras.

<img src="./recursos/doc2vec.jpg">

#### Ejemplo

**Paso 1** Entrenamiento del modelo

In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Configuración
#-------------
__longitud_vector = 5
__ventana = 10
__frecuencia_minima = 5
__learning_rate = 0.025
__min_learning_rate = 0.025
__epoch = 100
#---------------------

                
tagged_data = [TaggedDocument(words=word_tokenize(_d .lower()), tags=[str(i)]) for i, _d in enumerate(documentos)]
model = gensim.models.Doc2Vec(vector_size=__longitud_vector, window=__ventana, min_count=__frecuencia_minima, workers=11,alpha=__learning_rate, min_alpha=__min_learning_rate, epochs=__epoch)
model.build_vocab(tagged_data)
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

#Recorrido de todos los documentos con los que se ha entrenado
for i in range(20):
    print (model.docvecs[i].tolist())
    


[-2.5545856952667236, -1.5849534273147583, -0.1878330558538437, 1.073638677597046, -1.363091230392456]
[-2.392108917236328, -0.42625677585601807, 1.7279161214828491, 1.9812594652175903, -2.7751710414886475]
[2.4630959033966064, -0.8238241076469421, -2.142976760864258, 1.2622991800308228, -4.035261154174805]
[0.2854943871498108, -0.5213707089424133, -5.487168788909912, 2.7023210525512695, -2.681680202484131]
[-0.5289492607116699, -0.6331165432929993, -4.9254231452941895, 1.4784272909164429, -2.797506093978882]
[-1.6340490579605103, -3.5227088928222656, 0.0954757109284401, 1.7731825113296509, -4.2072367668151855]
[2.498589038848877, 2.6918082237243652, -0.28682583570480347, 1.4801366329193115, -2.7096784114837646]
[0.8234078288078308, -2.400024890899658, -2.570847272872925, 1.315682053565979, -2.9202115535736084]
[-0.8254379034042358, 2.2258379459381104, -2.2553091049194336, 1.5477888584136963, 0.08129897713661194]
[-5.665258407592773, -2.559307098388672, 2.945918321609497, 2.10811018943

**Paso 2** Cálculo de similitud

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

elemento =model.docvecs[45]

heap = []
for i in range(len(model.docvecs)):
    sim =cosine_similarity(elemento.reshape(1, -1), model.docvecs[i].reshape(1, -1))
    heapq.heappush(heap, (sim, documentos[i]))

print (documentos[45])
heapq.nlargest(10,heap)

Situada en el pueblo de La Zubia, junto al Parque Natural de Sierra Nevada y tan solo 5 min de la capital nazarí. Ideal tanto para hacer excursiones y practicar senderismo como para visitar La Alhambra y la ciudad. Fácil acceso por autovía A- 92 10 min a La Alhambra, 20 min a la estación de esquí de Sierra Nevada, 25 a la Alpujarra y a 30 la playa. Chimenea para relajarse al calor de la lumbre, salón amplio, dividido en dos ambientes, con ventana de acceso a la cocina y tv grande (ideal para reuniones grandes). La cocina dispone de todo lo necesario, patio andaluz, 12 dormitorios con mucha luz. Disponemos de restaurante especializado en carnes a la brasa, tanto el hotel como el restaurante están rodeados de jardines para tomar el sol en cualquier época del año.


[(array([[1.]], dtype=float32),
  'Situada en el pueblo de La Zubia, junto al Parque Natural de Sierra Nevada y tan solo 5 min de la capital nazarí. Ideal tanto para hacer excursiones y practicar senderismo como para visitar La Alhambra y la ciudad. Fácil acceso por autovía A- 92 10 min a La Alhambra, 20 min a la estación de esquí de Sierra Nevada, 25 a la Alpujarra y a 30 la playa. Chimenea para relajarse al calor de la lumbre, salón amplio, dividido en dos ambientes, con ventana de acceso a la cocina y tv grande (ideal para reuniones grandes). La cocina dispone de todo lo necesario, patio andaluz, 12 dormitorios con mucha luz. Disponemos de restaurante especializado en carnes a la brasa, tanto el hotel como el restaurante están rodeados de jardines para tomar el sol en cualquier época del año.'),
 (array([[0.99575377]], dtype=float32),
  'En pleno corazón del Parque Natural de Cazorla, Segura y Las Villas, como un tesoro escondido entre las montañas, se encuentra el Hotel Balneario P